In [315]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter



# Dataset preparation

In [316]:
#Pre-processes dataset and splits it into train and test-set in 7:3 ratio
def preprocessing(df, y_label, frac):

    #Drop rows with missing values (if any)
    df = df.dropna()

    #split into train and test set
    df_train = df.sample(frac=frac, axis=0)
    df_test = df.drop(df_train.index)

    #Choose target feature
    y_train = df_train[y_label].to_numpy()
    X_train = df_train.drop(columns=y_label).to_numpy()

    y_test = df_test[y_label].to_numpy()
    X_test = df_test.drop(columns=y_label).to_numpy()

    return X_train, y_train, X_test, y_test



In [317]:
#Car evaluation (https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) Target attribute: safety (low, med, high).
df_car = pd.read_csv('car+evaluation/car.data', sep=',', header=None)
Xtrain_car, ytrain_car, Xtest_car, ytest_car = preprocessing(df_car, 5, 0.7)


# Decision Tree Implementation

In [318]:
#Decision tree implementation inspired from: https://medium.com/@omidsaghatchian/decision-tree-implementation-from-scratch-visualization-5eb0bbf427c2

class Node:
    """
    Parameters
    ----------
    data: numpy.ndarray, default=None
        The dataset includes X and Y
    children: dict(feat_value: Node), default=None
        Dict of children
    split_on: int, default=None
        Index of the feature that node was split on that
    pred_class : str, default=None
        The predicted class for the node (only applicable to leaf nodes)
    is_leaf: bool, default=False
        Determine whether the node is leaf or not
    gain: float, default=None
        Information gained by performing split
    class_prob: dict(feat_value: class_prob), default=None
        dict of class probabilities


    """

    def __init__(self ,data=None, children=None, split_on = None, pred_class=None, is_leaf=False, gain=None, class_prob=None, entropy=None, gini=None):

        self.data = data
        self.children = children
        self.split_on = split_on
        self.pred_class = pred_class
        self.is_leaf = is_leaf
        self.gain = gain
        self.class_prob = class_prob
        self.entropy = entropy
        self.gini = gini



class DecisionTreeClassifier:

    def __init__(self, criterium, max_depth):
        self.root = Node()
        self.max_depth = max_depth
        #Impurity criterium used for splitting nodes
        self.criterium = criterium


    def calculate_entropy(self, y):
        """
        Calculates entropy of the given label array

        Parameters:
        -----------
        Y: numpy.ndarray
            The labels array.

        Returns:
        -----------
        entropy: flaot
            The entropy value of the given labels.

        """
        _, labels_counts = np.unique(y, return_counts=True)
        total_instances = len(y)
        entropy = sum([label_count / total_instances * np.log2(1 / (label_count / total_instances)) for label_count in labels_counts])
        return entropy


    def calculate_gini(self, y):
        """
        Calculate Gini index for array y of classes.
        """
        classes, counts = np.unique(y, return_counts=True)
        p = counts / counts.sum()
        gini = 1 - np.sum(p ** 2)
        return gini


    def calculate_class_probabilities(self, y):
        counts = Counter(y)
        total = len(y)
        return {cls: count / total for cls, count in counts.items()}


    def split_on_feature(self, data, feat_index, criterium):
        """
        Split the dataset based on a specific feature index.

        Parameters:
        -----------
        data: numpy.ndarray
            The dataset to be split.

        feat_index: int
            The index of the feature to perform the split.

        Returns:
        -----------
        - split_nodes: dict
            A dictionary of split nodes.
            (feature value as key, corresponding node as value)

        - weighted_entropy: float
            The weighted entropy of the split.
        """
        feature_values = data[:, feat_index]
        unique_values = np.unique(feature_values)

        split_nodes = {}
        weighted_impurity = 0
        total_instances = len(data)

        for unique_value in unique_values:
            partition = data[data[:, feat_index] == unique_value, :]
            node = Node(data=partition)
            split_nodes[unique_value] = node
            partition_y = partition[:, -1]
            #iteratively calculate entropy for each child node and use it to compute weighted entropy of the split
            if criterium == 'entropy':
                node_entropy = self.calculate_entropy(partition_y)
                weighted_impurity += (len(partition) / total_instances) * node_entropy
            else:
                node_gini = self.calculate_gini(partition_y)
                weighted_impurity += (len(partition) / total_instances) * node_gini

        return split_nodes, weighted_impurity


    def best_split(self, node, criterium, depth=0):
        """
        Find the best split for the given node.
        (data in node.data)

        Parameters:
        ----------
        node: Node
            The node for which the best split is being determined.

        If the node meets the criteria to stop splitting:
            - Mark the node as a leaf.
            - Assign a predicted class for future predictions based on the target values (y).
            - return.

        Otherwise:
            - Initialize variables for tracking the best split.
            - Iterate over the features to find the best split.
            - Split the data based on each feature and calculate the weighted entropy of the split.
            - Recursively call the best_split function for each child node.

        """

        y = self.get_y(node.data)
        #Impurity of current node
        if criterium == 'entropy':
            node_entropy = self.calculate_entropy(y)
            node.entropy = node_entropy
        else:
            node_gini = self.calculate_gini(y)
            node.gini = node_gini
        #Class probabilities
        node.class_prob = self.calculate_class_probabilities(y)


        # Check if the node meets the criteria to stop splitting
        if self.meet_criteria(node, depth):
            node.is_leaf = True
            y = self.get_y(node.data)
            node.pred_class = self.get_pred_class(y)
            return

        # Initialize variables for tracking the best split
        feature_split_index = -1
        min_impurity = float('inf')
        child_nodes = None


        # iterate over all features, ignore (y)
        for i in range(node.data.shape[1] - 1):
            split_nodes, weighted_impurity = self.split_on_feature(node.data, i, criterium)
            if weighted_impurity < min_impurity:
                child_nodes, min_impurity = split_nodes, weighted_impurity
                feature_split_index = i



        node.children = child_nodes
        node.split_on = feature_split_index

        #calculate information gain gained by splitting
        if criterium == 'entropy':
            node.gain = node_entropy -  min_impurity
        else:
            node.gain = node_gini - min_impurity


        # Recursively call the best_split function for each child node
        for child_node in child_nodes.values():
            self.best_split(child_node, criterium, depth + 1)


    #Stopping criteria
    def meet_criteria(self, node, depth):
        """
        Check if maximum depth has been reached or entropy calculated is 0

        Parameters:
        -----------
        node : Node
            The node to check for meeting the stopping criteria.

        Returns:
        -----------
        bool
            True if the criteria is met, False otherwise.

        """

        y = self.get_y(node.data)
        return True if self.calculate_entropy(y) == 0 or depth >= self.max_depth else False


    def get_y(self, data):
        """
        Get the target (y) from the data.

        Parameters:
        -----------
        data : numpy.ndarray
            The input data containing features and the target variable.

        Returns:
        -----------
        y: numpy.ndarray
            The target variable extracted from the data.

        """
        y = data[:, -1]
        return y


    def get_pred_class(self, y):
        """
        Get the predicted class label based on the majority vote.

        Parameters:
        -----------
        Y : numpy.ndarray
            The array of class labels.

        Returns:
        -----------
        str
            The predicted class label.

        """

        labels, labels_counts = np.unique(y, return_counts=True)
        index = np.argmax(labels_counts)
        return labels[index]

    def fit(self, X, y):
        """
        Fit the decision tree model to the provided dataset.

        Parameters:
        -----------
        X: numpy.ndarray
            The input features of the dataset.

        y: numpy.ndarray
            The target labels of the dataset.
        """
        data = np.column_stack([X, y])
        self.root.data = data
        self.best_split(self.root, self.criterium, depth=0)

    def predict(self, X):
        """
        Predict the class labels for the given input features.

        Parameters:
        -----------
        X: numpy.ndarray
            The input features for which to make predictions. Should be a 2D array-like object.

        Returns:
        -----------
        predictions: numpy.ndarray
            An array of predicted class labels.

        """

        # Traverse the decision tree for each input and make predictions
        predictions = np.array([self.traverse_tree(x, self.root) for x in X])
        return predictions

    def traverse_tree(self, x, node):
        """
        Recursively traverse the decision tree to predict the class label for a given input.

        Parameters:
        -----------
        x:
            The input for which to make a prediction.

        node:
            The current node being traversed in the decision tree.

        Returns:
        -----------
        predicted_class:
            The predicted class label for the input feature.

        """

        # Check if the current node is a leaf node
        if node.is_leaf:
            return node.pred_class

        # Get the feature value at the split point for the current node
        feat_value = x[node.split_on]

         # Handle unseen feature values
        if feat_value not in node.children:
            return node.pred_class  # fallback to majority class at this node

        # Recursively traverse the decision tree using the child node corresponding to the feature value
        predicted_class = self.traverse_tree(x, node.children[feat_value])

        return predicted_class


    #Pretty prints tree using ASCII characters. Each node displays Impurity, gain, depth, class probabilities and feature on which the node was split
    def print_tree(self, node=None, prefix="", is_last=True, depth=0):
        if node is None:
            node = self.root

        connector = "└── " if is_last else "├── "
        print(prefix + connector, end="")

        # Format class probabilities string if available
        if node.class_prob:
            probs_str = ", ".join([f"{cls}: {prob:.2f}" for cls, prob in node.class_prob.items()])
        else:
            probs_str = "No class probabilities"

        # Choose correct impurity label
        if node.entropy is not None:
            impurity_str = f"Entropy: {node.entropy:.4f}"
        elif node.gini is not None:
            impurity_str = f"Gini: {node.gini:.4f}"
        else:
            impurity_str = "Impurity: N/A"

        # Leaf or internal node output
        if node.is_leaf:
            print(f"[Leaf] Depth: {depth}, Class: {node.pred_class}, {impurity_str}, Probs: {{{probs_str}}}")
        else:
            print(f"X[{node.split_on}], Gain: {node.gain:.4f}, Depth: {depth}, {impurity_str}, Probs: {{{probs_str}}}")

        # Recursively print children
        if node.children:
            prefix += "    " if is_last else "│   "
            child_count = len(node.children)
            for i, (feat_val, child) in enumerate(node.children.items()):
                is_last_child = (i == child_count - 1)
                print(prefix + ("└── " if is_last_child else "├── ") + f"X[{node.split_on}] = {feat_val}")
                self.print_tree(child, prefix + ("    " if is_last_child else "│   "), True, depth + 1)

    def predict_probs(self, X):
        """
        Predict class probabilities for each instance in X.

        Parameters:
        -----------
        X: numpy.ndarray
            Feature matrix (num_samples, num_features)

        Returns:
        -----------
        probs: List[Dict]
            A list where each item is a dictionary mapping class labels to probabilities.
        """
        probs = []
        for instance in X:
            node = self.root
            while not node.is_leaf:
                feat_val = instance[node.split_on]
                if feat_val in node.children:
                    node = node.children[feat_val]
                else:
                    # If feature value not seen in training, break and use current node
                    break
            probs.append(node.class_prob)
        return probs


#Using class level labels to calculate cross entropy loss
def cross_entropy_loss(y_true, y_pred, eps=1e-9):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    loss = 0
    for yt, yp in zip(y_true, y_pred):
        p = 1 - eps if yt == yp else eps
        loss += -np.log(p)
    return loss / len(y_true)

#Cross entropy calculated using predicted probabilities
def true_cross_entropy_loss(y_true, y_pred_probs, eps=1e-9):
    loss = 0
    for true, probs_dict in zip(y_true, y_pred_probs):
        p = probs_dict.get(true, 0)
        loss += -np.log(p + eps)
    return loss / len(y_true)


def accuracy(y_true, y_pred):
    return np.mean(np.array(y_true) == np.array(y_pred))


Instead of printing histograms for each node, for the sake of readability we print the class probabilities and impurity measure in the tree within the node itself.

We start by building and printing the decision tree for the car dataset using Cross entropy as the quality measure.

In [319]:
model = DecisionTreeClassifier("entropy", max_depth=3)
model.fit(Xtrain_car, ytrain_car)

print("ROOT")
model.print_tree(model.root)

#predicting labels using the test set
y_pred = model.predict(Xtest_car)

loss = cross_entropy_loss(ytest_car, y_pred)
true_loss = true_cross_entropy_loss(ytest_car, model.predict_probs(Xtest_car))
acc = accuracy(ytest_car, y_pred)

print(f"\nCross Entropy Loss using Hard Labels: {loss:.4f}")
print(f"Cross Entropy Loss using Probabilities: {true_loss:.4f}")
print(f"Accuracy on Test-Set: {acc * 100:.2f}%")


ROOT
└── X[5], Gain: 0.2731, Depth: 0, Entropy: 1.5846, Probs: {low: 0.34, high: 0.33, med: 0.32}
    ├── X[5] = acc
    │   └── X[0], Gain: 0.0352, Depth: 1, Entropy: 0.9806, Probs: {high: 0.58, med: 0.42}
    │       ├── X[0] = high
    │       │   └── X[4], Gain: 0.1530, Depth: 2, Entropy: 0.8941, Probs: {high: 0.69, med: 0.31}
    │       │       ├── X[4] = big
    │       │       │   └── [Leaf] Depth: 3, Class: high, Entropy: 0.9940, Probs: {med: 0.45, high: 0.55}
    │       │       ├── X[4] = med
    │       │       │   └── [Leaf] Depth: 3, Class: high, Entropy: 0.9183, Probs: {high: 0.67, med: 0.33}
    │       │       └── X[4] = small
    │       │           └── [Leaf] Depth: 3, Class: high, Entropy: 0.0000, Probs: {high: 1.00}
    │       ├── X[0] = low
    │       │   └── X[1], Gain: 0.3027, Depth: 2, Entropy: 0.9880, Probs: {med: 0.56, high: 0.44}
    │       │       ├── X[1] = high
    │       │       │   └── [Leaf] Depth: 3, Class: med, Entropy: 0.9044, Probs: {med: 0.68,

Next the decision tree is built using the Gini index as the quality criterium

In [320]:
model = DecisionTreeClassifier("gini", max_depth=3)
model.fit(Xtrain_car, ytrain_car)

print("ROOT")
model.print_tree(model.root)

#predicting labels using the test set
y_pred = model.predict(Xtest_car)

loss = cross_entropy_loss(ytest_car, y_pred)
true_loss = true_cross_entropy_loss(ytest_car, model.predict_probs(Xtest_car))
acc = accuracy(ytest_car, y_pred)

print(f"\nCross Entropy Loss using Hard Labels: {loss:.4f}")
print(f"Cross Entropy Loss using Probabilities: {true_loss:.4f}")
print(f"Accuracy on Test-Set: {acc * 100:.2f}%")


ROOT
└── X[5], Gain: 0.0985, Depth: 0, Gini: 0.6665, Probs: {low: 0.34, high: 0.33, med: 0.32}
    ├── X[5] = acc
    │   └── X[0], Gain: 0.0236, Depth: 1, Gini: 0.4866, Probs: {high: 0.58, med: 0.42}
    │       ├── X[0] = high
    │       │   └── X[4], Gain: 0.0631, Depth: 2, Gini: 0.4284, Probs: {high: 0.69, med: 0.31}
    │       │       ├── X[4] = big
    │       │       │   └── [Leaf] Depth: 3, Class: high, Gini: 0.4959, Probs: {med: 0.45, high: 0.55}
    │       │       ├── X[4] = med
    │       │       │   └── [Leaf] Depth: 3, Class: high, Gini: 0.4444, Probs: {high: 0.67, med: 0.33}
    │       │       └── X[4] = small
    │       │           └── [Leaf] Depth: 3, Class: high, Gini: 0.0000, Probs: {high: 1.00}
    │       ├── X[0] = low
    │       │   └── X[1], Gain: 0.1691, Depth: 2, Gini: 0.4917, Probs: {med: 0.56, high: 0.44}
    │       │       ├── X[1] = high
    │       │       │   └── [Leaf] Depth: 3, Class: med, Gini: 0.4352, Probs: {med: 0.68, high: 0.32}
    │      

The loss and accuracy for the trees built using the cross entropy and gini index criteria are identical

# Pruning the Decision Tree

In [321]:
def prune(node, X_val, y_val, model):
    if node.is_leaf or not node.children:
        return

    for child in node.children.values():
        prune(child, X_val, y_val, model)

    #calculate accuracy of tree pre-pruning current node
    val_pred_original = model.predict(X_val)
    acc_original = accuracy(y_val, val_pred_original)

    # Save state
    original_children = node.children
    original_is_leaf = node.is_leaf
    original_pred = node.pred_class

    # Try pruning
    node.children = None
    node.is_leaf = True
    node.pred_class = model.get_pred_class(model.get_y(node.data))

    # Evaluate performance on validation set after pruning
    val_pred_pruned = model.predict(X_val)
    acc_pruned = accuracy(y_val, val_pred_pruned)


    # Restore original tree if accuracy better before pruning
    if acc_pruned < acc_original:
        node.children = original_children
        node.is_leaf = original_is_leaf
        node.pred_class = original_pred

#Split data into train, test and validation set in the ratio of 34:33:33
def train_test_val(df, y_label):

    #Drop rows with missing values (if any)
    df = df.dropna()

    #split into train and temp set
    df_train = df.sample(frac=0.34, axis=0)
    df_temp = df.drop(df_train.index)

    df_val = df_temp.sample(frac=0.5, axis=0)
    df_test = df_temp.drop(df_val.index)

    #Choose target feature
    y_train = df_train[y_label].to_numpy()
    X_train = df_train.drop(columns=y_label).to_numpy()

    y_val = df_val[y_label].to_numpy()
    X_val = df_val.drop(columns=y_label).to_numpy()

    y_test = df_test[y_label].to_numpy()
    X_test = df_test.drop(columns=y_label).to_numpy()

    return X_train, y_train, X_val, y_val, X_test, y_test


The decision tree is built using cross entropy as the quality measure and maximum depth 10 as the stopping criterium. The tree is then pruned.

In [322]:
#Split the dataset into test, train and validation sets
Xtrain_car, ytrain_car, X_val_car, y_val_car, X_test_car, y_test_car = train_test_val(df_car,5)

model = DecisionTreeClassifier("entropy", max_depth=10)
model.fit(Xtrain_car, ytrain_car)

#Pruning using validation set
prune(model.root, X_val_car, y_val_car, model)

print("ROOT")
model.print_tree(model.root)

# Calculate performance of the pruned tree on the test set
y_pred = model.predict(X_test_car)

loss = cross_entropy_loss(y_test_car, y_pred)
true_loss = true_cross_entropy_loss(y_test_car, model.predict_probs(X_test_car))
acc = accuracy(y_test_car, y_pred)

print(f"\nCross Entropy Loss using Hard Labels (Post-Pruning): {loss:.4f}")
print(f"Cross Entropy Loss using Probabilities (Post-Pruning): {true_loss:.4f}")
print(f"Accuracy on Test-Set (Post-Pruning): {acc * 100:.2f}%")

ROOT
└── X[5], Gain: 0.2830, Depth: 0, Entropy: 1.5838, Probs: {high: 0.34, low: 0.34, med: 0.31}
    ├── X[5] = acc
    │   └── X[4], Gain: 0.0209, Depth: 1, Entropy: 0.9683, Probs: {high: 0.60, med: 0.40}
    │       ├── X[4] = big
    │       │   └── [Leaf] Depth: 2, Class: med, Entropy: 0.9996, Probs: {high: 0.49, med: 0.51}
    │       ├── X[4] = med
    │       │   └── X[2], Gain: 0.0700, Depth: 2, Entropy: 0.9389, Probs: {high: 0.64, med: 0.36}
    │       │       ├── X[2] = 2
    │       │       │   └── X[0], Gain: 0.2365, Depth: 3, Entropy: 0.7219, Probs: {high: 0.80, med: 0.20}
    │       │       │       ├── X[0] = high
    │       │       │       │   └── [Leaf] Depth: 4, Class: high, Entropy: 0.0000, Probs: {high: 1.00}
    │       │       │       ├── X[0] = low
    │       │       │       │   └── [Leaf] Depth: 4, Class: high, Entropy: 0.0000, Probs: {high: 1.00}
    │       │       │       ├── X[0] = med
    │       │       │       │   └── X[1], Gain: 0.4200, Depth: 4, Ent

Compared to the accuracy of the decision tree built wihtout pruning, the pruned tree performs consistently better.